In [193]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import sys

import tensorflow as tf
from tensorflow import keras
import seaborn as sns
from pylab import rcParams
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters

In [194]:
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv

from keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam


from tensorflow.keras import layers, regularizers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Flatten, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.layers import Conv2D, MaxPool2D, LeakyReLU
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import shuffle
print(tf.__version__)

2.1.0


In [195]:
df = pd.read_csv('acc_data (right_wrist).csv')
X = np.column_stack((np.array(df['Acc_X']), np.array(df['Acc_Y']), np.array(df['Acc_Z'])))
y = np.array(df['PacketCounter']).reshape(-1, 1)

In [196]:
columns =['PacketCounter', 'Acc_X', 'Acc_Y', 'Acc_Z']
df = pd.DataFrame(data = df, columns = columns)
df.head()

,PacketCounter,Acc_X,Acc_Y,Acc_Z
0,0,0.000000,0.000000,0.000000
1,1,-8.874124,-0.679627,2.768368
2,2,-8.873473,-0.613973,2.761706
3,3,-8.834931,-0.518206,2.797576
4,4,-8.921975,-0.508252,2.836315


In [197]:
df.shape

(403, 4)

In [198]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403 entries, 0 to 402
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PacketCounter  403 non-null    int64  
 1   Acc_X          403 non-null    float64
 2   Acc_Y          403 non-null    float64
 3   Acc_Z          403 non-null    float64
dtypes: float64(3), int64(1)
memory usage: 12.7 KB


In [199]:
df.isnull().sum()

PacketCounter    0
Acc_X            0
Acc_Y            0
Acc_Z            0
dtype: int64

In [200]:
#df['Acc_X'].value_counts()

In [201]:
#df['Acc_Y'].value_counts()

In [202]:
#df['Acc_Z'].value_counts()

In [203]:
df['Acc_X'] = df['Acc_X'].astype('float')
df['Acc_Y'] = df['Acc_Y'].astype('float')
df['Acc_Z'] = df['Acc_Z'].astype('float')

In [204]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403 entries, 0 to 402
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PacketCounter  403 non-null    int64  
 1   Acc_X          403 non-null    float64
 2   Acc_Y          403 non-null    float64
 3   Acc_Z          403 non-null    float64
dtypes: float64(3), int64(1)
memory usage: 12.7 KB


In [205]:
label = LabelEncoder()
df['label'] = label.fit_transform(df['PacketCounter'])
df.head()

,PacketCounter,Acc_X,Acc_Y,Acc_Z,label
0,0,0.000000,0.000000,0.000000,0
1,1,-8.874124,-0.679627,2.768368,1
2,2,-8.873473,-0.613973,2.761706,2
3,3,-8.834931,-0.518206,2.797576,3
4,4,-8.921975,-0.508252,2.836315,4


In [206]:
X = df[['Acc_X', 'Acc_Y', 'Acc_Z']]
y = df['label']

In [207]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

scaled_X = pd.DataFrame(data = X, columns = ['Acc_X', 'Acc_Y', 'Acc_Z'])
scaled_X['label'] = y.values

scaled_X.head()

,Acc_X,Acc_Y,Acc_Z,label
0,1.065724,-0.010709,-1.222835,0
1,-0.378791,-0.153964,-0.015481,1
2,-0.378685,-0.140125,-0.018386,2
3,-0.372411,-0.119939,-0.002742,3
4,-0.386580,-0.117841,0.014152,4


In [208]:
import scipy.stats as stats
Fs = 1
frame_size = Fs*2 # 80
hop_size = Fs*1 # 40


In [209]:
def get_frames(df, frame_size, hop_size):

    N_FEATURES = 3

    frames = []
    labels = []
    for i in range(0, len(df) - frame_size, hop_size):
        x = df['Acc_X'].values[i: i + frame_size]
        y = df['Acc_Y'].values[i: i + frame_size]
        z = df['Acc_Z'].values[i: i + frame_size]
        
        # Retrieve the most often used label in this segment
        label = stats.mode(df['label'][i: i + frame_size])[0][0]
        frames.append([x, y, z])
        labels.append(label)

    # Bring the segments into a better shape
    frames = np.asarray(frames).reshape(-1, frame_size, N_FEATURES)
    labels = np.asarray(labels)

    return frames, labels

X, y = get_frames(scaled_X, frame_size, hop_size)

X.shape, y.shape

((401, 2, 3), (401,))

In [212]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [213]:
X_train.shape, X_test.shape

((320, 2, 3), (81, 2, 3))

In [214]:
X_train[0].shape, X_test[0].shape

((2, 3), (2, 3))

In [228]:
X_train = X_train.reshape(320, 2, 3, 1)
X_test = X_test.reshape(81, 2, 3, 1)

In [229]:
X_train[0].shape, X_test[0].shape

((2, 3, 1), (2, 3, 1))

In [230]:
# fit and evaluate a model
def evaluate_model(X_train, y_train, X_test, y_test):
	verbose, epochs, batch_size = 0, 15, 64
	n_timesteps, n_features, n_outputs = X_train.shape[0], X_train.shape[1], y_train.shape[0]
	model = Sequential()
	model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
	model.add(Dropout(0.5))
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
	return accuracy

In [231]:
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))


In [232]:
# run an experiment
def run_experiment():#repeats=10):

    scores = list()
    #for r in range(repeats):
    score = evaluate_model(X_train, y_train, X_test, y_test)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
    # summarize results
    summarize_results(scores)

In [233]:
# run the experiment
run_experiment()

ValueError: Error when checking input: expected lstm_9_input to have 3 dimensions, but got array with shape (320, 2, 3, 1)